<a href="https://colab.research.google.com/github/kumiori/mec647/blob/main/mec647_Latest_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
%%time
%%capture
import sys

try:
    import google.colab  # noqa: F401
except ImportError:
    import ufl  # noqa: F401
    import dolfinx  # noqa: F401
else:
    try:
        import ufl
        import dolfinx
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/fenicsx-install.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh";
        import ufl  # noqa: F401
        import dolfinx  # noqa: F401

try:
    import pyvista
except ImportError:
    !{sys.executable} -m pip install --upgrade pyvista itkwidgets;
    import pyvista  # noqa: F401
    from pyvista.utilities import xvfb

try:
    import gmsh
except ImportError:
    !{sys.executable} -m pip install gmsh
    import gmsh

!sudo apt install libgl1-mesa-glx xvfb;
!{sys.executable} -m pip install pythreejs;
!{sys.executable} -m pip install ipygany;
!{sys.executable} -m pip install --upgrade PyYAML

In [ ]:
import matplotlib.pyplot as plt
from petsc4py import PETSc
import numpy as np
from mpi4py import MPI
comm = MPI.COMM_WORLD

branch_name = 'main'

!rm -rf mec647
try:
  !git clone -b {branch_name} https://github.com/kumiori/mec647.git
  sys.path.append('mec647/')

  import mec647
  from mec647 import meshes
  from mec647.meshes import primitives
  from mec647.utils.viz import plot_mesh, plot_scalar, plot_vector

except Exception as e:
  print('Something went wrong', e)
  !rm -rf mec647
  !git clone https://github.com/kumiori/mec647.git



In [ ]:
%%capture
from dolfinx.io import XDMFFile
from mec647.meshes import gmsh_model_to_mesh
from mpi4py import MPI
from pathlib import Path
gmsh.finalize()

# We parametrise the domain (target) dimension, we solve 2D for now 
tdim = 2

# Default mesh parameters are geometric

if branch_name == 'andres-plates':
  from mec647.meshes.ikea import mesh_ikea_real as slab_with_holes

  model, tdim, tag_names = mesh_function('perforated_slab',
                  geom_parameters={},
                  lc=.01,
                  tdim=2,
                  order=0,
                  msh_file='perforated_slab.msh'
                  )
elif branch_name == 'main':
  from mec647.meshes import primitives

  default_parameters = {
      'geometry': {
          'geom_type': 'bar',
          'Lx': 1.,
          'Ly': 0.1
      }
  }


  gmsh_model, tdim = primitives.mesh_bar_gmshapi(
      default_parameters.get('geometry').get('geom_type'),
      default_parameters.get('geometry').get('Lx'), 
      default_parameters.get('geometry').get('Ly'), 
      lc=0.03, 
      tdim=tdim)

mesh, mts = meshes.gmsh_model_to_mesh(gmsh_model,
                               cell_data=False,
                               facet_data=True,
                               gdim=2)




# mesh, mts = gmsh_model_to_mesh(
#     model, cell_data=True, facet_data=False, gdim=tdim)


In [ ]:
# Viz the mesh

plt.figure()
ax = plot_mesh(mesh)
fig = ax.get_figure()
plt.title(f"Mesh with parameters, dimension {tdim}")
# fig.savefig(f"one_mesh.png")

In [ ]:
# Let's get the entire set of parameters
import yaml

with open("mec647/test/parameters.yml") as f:
    parameters = yaml.load(f, Loader=yaml.FullLoader)
    

In [ ]:
# Functional Setting
import basix.ufl

element_u = basix.ufl.element("Lagrange", mesh.basix_cell(),
                              degree=1, shape=(2,))

element_alpha = basix.ufl.element("Lagrange", mesh.basix_cell(),
                              degree=1)

V_u = dolfinx.fem.functionspace(mesh, element_u) 
V_alpha = dolfinx.fem.functionspace(mesh, element_alpha) 

u = dolfinx.fem.Function(V_u, name="Displacement")
u_ = dolfinx.fem.Function(V_u, name="BoundaryDisplacement")


alpha = dolfinx.fem.Function(V_alpha, name="Damage")

# Pack state
state = {"u": u, "alpha": alpha}

# Bounds
alpha_ub = dolfinx.fem.Function(V_alpha, name="UpperBoundDamage")
alpha_lb = dolfinx.fem.Function(V_alpha, name="LowerBoundDamage")

dx = ufl.Measure("dx", domain = mesh)
ds = ufl.Measure("ds", domain = mesh)


# Useful references
Lx = parameters.get("geometry").get("Lx")
Ly = parameters.get("geometry").get("Ly")

In [ ]:
# Boundary sets

from dolfinx.fem import locate_dofs_geometrical, dirichletbc


dofs_alpha_left = locate_dofs_geometrical(V_alpha, lambda x: np.isclose(x[0], 0.))
dofs_alpha_right = locate_dofs_geometrical(V_alpha, lambda x: np.isclose(x[0], Lx))

dofs_u_left = locate_dofs_geometrical(V_u, lambda x: np.isclose(x[0], 0.))
dofs_u_right = locate_dofs_geometrical(V_u, lambda x: np.isclose(x[0], Lx))

# Boundary data

u_.interpolate(lambda x: (np.ones_like(x[0]), np.zeros_like(x[1])))

# Bounds (nontrivial)

alpha_lb.interpolate(lambda x: np.zeros_like(x[0]))
alpha_ub.interpolate(lambda x: np.ones_like(x[0]))

In [ ]:
# Boundary conditions

bcs_u = [
         dirichletbc(np.array([0., 0.], dtype=PETSc.ScalarType),
                      dofs_u_left,
                      V_u),
         dirichletbc(u_, dofs_u_right)
         ]

bcs_alpha = [
             dirichletbc(np.array(0., dtype = PETSc.ScalarType),
                         np.concatenate([dofs_alpha_left, dofs_alpha_right]),
                         V_alpha)
]
bcs = {"bcs_u": bcs_u, "bcs_alpha": bcs_alpha}


In [ ]:
# Material behaviour
from mec647.models import DamageElasticityModel as Brittle

model = Brittle(parameters["model"])

total_energy = model.total_energy_density(state) * dx


In [ ]:
# Evolution solver
import algorithms
from algorithms import am

solver = am.AlternateMinimisation(total_energy,
                         state,
                         bcs,
                         parameters.get("solvers"),
                         bounds=(alpha_lb, alpha_ub)
                         )

In [ ]:
%%time
# Loop for evolution
from dolfinx.fem import assemble_scalar


loads = np.linspace(parameters.get("loading").get("min"),
                    parameters.get("loading").get("max"),
                    parameters.get("loading").get("steps"))

data = {
    'elastic': [],
    'surface': [],
    'total': [],
    'load': []
}
for (i_t, t) in enumerate(loads):
  # update boundary conditions

  u_.interpolate(lambda x: (t * np.ones_like(x[0]), np.zeros_like(x[1])))
  u_.x.petsc_vec.ghostUpdate(addv=PETSc.InsertMode.INSERT,
                        mode=PETSc.ScatterMode.FORWARD)

  # update lower bound for damage
  alpha.x.petsc_vec.copy(alpha_lb.x.petsc_vec)
  alpha.x.petsc_vec.ghostUpdate(addv=PETSc.InsertMode.INSERT,
                        mode=PETSc.ScatterMode.FORWARD)

  # solve for current load step
  print(f"Solving timestep {i_t}, load: {t}")

  solver.solve()

  # postprocessing
  # global

  surface_energy = comm.allreduce(
          assemble_scalar(
            dolfinx.fem.form(model.damage_dissipation_density(state) * dx)),
          op=MPI.SUM
          )

  elastic_energy = comm.allreduce(
      assemble_scalar(
        dolfinx.fem.form(model.elastic_energy_density(state) * dx)),
          op=MPI.SUM
        )
  
  data.get('elastic').append(elastic_energy)
  data.get('surface').append(surface_energy)
  data.get('total').append(surface_energy+elastic_energy)
  data.get('load').append(t)

  print(f"Solved timestep {i_t}, load: {t}")
  print(f"Elastic Energy {elastic_energy:.3g}, Surface energy: {surface_energy:.3g}")
  print("\n\n")

  # savings?


In [ ]:
plt.plot(data.get('load'), data.get('surface'), label='surface', marker='o')
plt.plot(data.get('load'), data.get('elastic'), label='elastic', marker='o')
plt.plot(data.get('load'), [1./2. * t**2*Ly for t in data.get('load')], label='anal elast', ls=':', c='k')

plt.title('Traction bar energetics')
plt.legend()
plt.yticks([0, 1/20], [0, '$1/2.\sigma_c^2/E_0$'])
plt.xticks([0, 1], [0, 1])

In [ ]:
# experiments
params0, data0 = parameters, data


In [ ]:
from mec647.utils.viz import plot_vector, plot_scalar, plot_profile
plotter = pyvista.Plotter(
    title="Damage profile",
    window_size=[800, 600],
    shape=(1, 1),
)
_ell = parameters.get('model').get('ell')
tol = 1e-3
xs = np.linspace(0 + tol, Lx - tol, 101)
points = np.zeros((3, 101))
points[0] = xs

_plt, data = plot_profile(
    alpha,
    points,
    plotter,
    subplot=(0, 0),
    lineproperties={
        "c": "k",
        "label": f"$\alpha$ with $\ell$ = {_ell:.2f}"
    },
)
ax = _plt.gca()
ax.axvline(Lx/2 - 2*_ell, c="k")
ax.axvline(Lx/2 + 2*_ell, c="k", label='|supp|=$4\ell$')
ax.axvline(Lx/2 , c="k", ls=':', label='$x_0$')
_plt.legend()
_plt.fill_between(data[0], data[1].reshape(len(data[1])))
_plt.title("Traction bar damage profile")
